In [1]:
import numpy as np
import pandas as pd
from pandas import ExcelWriter
import math
from datetime import date

In [ ]:
#set thresholds for qualifying markets
pen = .095
cnts = 25000
#read in zips and fips file
sample1 = pd.read_excel("",skiprows=range(0,2),index_col=None, na_values=['0'], converters={'FIPS':str})
sample2 = pd.read_excel("",skiprows=range(0,2),index_col=None, na_values=['0'], converters={'FIPS':str, 'Zip Code':str})
last_year = pd.read_excl("", nan_values='')
last_year.reset_index(inplace=True,drop=True)
last_year = last_year.replace(np.nan,"",regex=True)

In [ ]:
#Name output file and add in market codes
today = date.today()
data_date = today.strftime("%b-%d-%Y")
filename = 'output'+data_date+'.xlsx'

market_dict = {'Atlanta, GA':001,'Beamont-Port Arthur, TX':002, ...}
#create dataframe and include other defined metros
allen_fipsum = sample1.loc[sample1['FIPS'].isin(["",""])]
allen_fipsum['mkt_name'] = 'Allentown'

target = sample1.append(allen_fipsum,ignore_index=True)
target['mkt'] = target['mkt_name'].map(mkt_dict)

In [ ]:
#pivot and calculate penetrations for black and hispanic
final_pivot = pd.pivot_table(target,values=["Tot,18+","Tot. Blk","Tot. Hisp"],index=['mkt','mkt_name'], aggfunc='sum')
final_pivot.reset_index(inplace=True)
final_pivot['BlkPen'] = (final_pivot['Tot. Blk']/final_piovt['Tot. 18+']).round(5)
final_pivot['HispPen'] = (final_pivot['Tot. Hips']/final_piovt['Tot. 18+']).round(5)

In [ ]:
#flags for qualifying markets and move other metros
condition_blk = [(final_pivot['BlkPen']>=pen), (final_pivot['Tot. Blk'] >= cnts)]
condition_hisp = [(final_pivot['HispPen']>=pen), (final_pivot['Tot. Hisp'] >= cnts)]
flag_race = ['*','*']
final_pivot['Blk qualify'] = np.select(condition_blk, flag_race, default="")
final_pivot['Hisp qualify'] = np.select(condition_hisp, flag_race, default="")

condition_mkt = (final_pivot.mkt_name=="Allentown")
in_order = final_pivot[~condition_mtkk].sort_values(by='mkt_name')
final_df = pd.concat([in_order, final_pivot[condition_mkt])
final_df.reset_index(inplace=True,drop=True)

In [ ]:
#compare to last year
final_df['match_blk'] = np.where(final_df['Blk qualify'] == last_year['Blk qualify'],0,1)
final_df['match_hisp'] = np.where(final_df['Hisp qualify'] == last_year['Hisp qualify'],0,1)

In [ ]:
print(final_df[final_df['match_hisp']==1])
print("********")
print("********")
print("********")
print(final_df[final_df['match_hisp']==1])

In [ ]:
#create final file
output = final_df[['mkt','mkt_name','BlkPen','Tot. Blk', 'Blk qualify','HispPen','Tot. Hisp', 'Hisp qualify', 'match_blk','match_hisp']]
writer = ExcelWriter(filename)
output.to_excel(writer,'Sheet1', index=False)
writer.save()
print('done')